In [ ]:
import gymnasium as gym
import simple_env
from simple_env.wrappers import RelativePosition, NormalizedObservation, DiscreteActions
from gymnasium.wrappers import TimeAwareObservation
from stable_baselines3 import DQN, A2C, PPO, DDPG
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback, BaseCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
import shutil
from typing import Callable
import numpy as np

name = '01_gridworld'
env_name = "simple_env/ContinuousWorld-v3"
world_size = 300
max_episode_steps = 50
action_mode = "continuous"

agent = PPO
policy = 'MlpPolicy'
dir = f"./{name}"
tensorboard_log = f"./{name}/t_logs/"
model_path = f"./{name}/model/best_model.zip"
best_model_save_path = f"./{name}/model/"
log_path = f"./{name}/logs/"
device = 'cpu'

# num_action = 20
# keys = [i for i in range(num_action)]
# values = [(i/num_action)*2 - 1 for i in range(num_action)]
# disc_to_cont = dict(zip(keys, values))
def make_env(render_mode = None):
    env = gym.make(
        env_name,
        render_mode = render_mode,
        size=world_size, 
        max_episode_steps=max_episode_steps,
        action_mode = action_mode
        )
    
    #env = DiscreteActions(env, disc_to_cont)
    #env = TimeAwareObservation(env, normalize_time=True)
    return env

    

# Test

In [ ]:
# Execute Setup
import nbformat
from IPython import get_ipython
with open("02_continuous_world.ipynb", "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)
for cell in notebook.cells:
    if "tags" in cell.metadata and "setup" in cell.metadata.tags:
        exec(cell.source)

env = make_env(render_mode='human')
obs, info = env.reset()

terminated = False
truncated = False

while not terminated and not truncated:
    
    action = env.action_space.sample()
    #action = np.random.randint(num_action)
    #action = 0
    obs, rew, terminated, truncated, info = env.step(action)
    print(f"Action: {action}, Obs: {obs}, rew: {rew}")
    
env.close()

# Create agent



In [ ]:
# Execute Setup
import nbformat
from IPython import get_ipython
with open("02_continuous_world.ipynb", "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)
for cell in notebook.cells:
    if "tags" in cell.metadata and "setup" in cell.metadata.tags:
        exec(cell.source)

# Linear Schedule
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    def func(progress_remaining: float) -> float:
        return progress_remaining * initial_value
    return func

# Create model
env = make_env()


if agent is DQN:
    model = DQN(
        policy,
        env,
        verbose=0,
        device=device,
        tensorboard_log=tensorboard_log,
        exploration_fraction=0.5,
        learning_rate=linear_schedule(0.0001)
    )
    
elif agent is DDPG:
    model = DDPG(
        policy,
        env,
        verbose=0,
        device=device,
        tensorboard_log=tensorboard_log,
        learning_rate=linear_schedule(0.0001)
    )
    
elif agent is A2C:
    model = A2C(
        policy,
        env,
        verbose=0,
        device=device,
        tensorboard_log=tensorboard_log,    
    )
    
elif agent is PPO:
    model = PPO(
        policy,
        env,
        verbose=0,
        device=device,
        tensorboard_log=tensorboard_log,    
    )

# Save
shutil.rmtree(dir, ignore_errors=True)
model.save(model_path)

# Train

In [ ]:
# Execute Setup
import nbformat
from IPython import get_ipython
with open("02_continuous_world.ipynb", "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)
for cell in notebook.cells:
    if "tags" in cell.metadata and "setup" in cell.metadata.tags:
        exec(cell.source)
        
print(f"Start training with {agent}")
# Env and model
train_env = make_env()
eval_env = make_env()

model = agent.load(model_path, train_env, device)

# Callbacks
eval_callback = EvalCallback(
    eval_env,
    eval_freq=1e4,
    deterministic=True,
    n_eval_episodes=10,
    best_model_save_path=best_model_save_path,
)

checkpoint_callback = CheckpointCallback(
    1e4,
    best_model_save_path,
    name_prefix="checkpoint"
)

# Training
model.learn(
    total_timesteps=5e7,
    progress_bar=True,
    reset_num_timesteps=False,
    
    callback=[
        eval_callback,
        checkpoint_callback
    ]
)

# Save and close
model.save(model_path)
train_env.close()

Output()

Start training with <class 'stable_baselines3.ppo.ppo.PPO'>


Eval num_timesteps=1800000, episode_reward=-0.06 +/- 0.63

Episode length: 46.20 +/- 11.40

New best mean reward!

...

Episode length: 50.00 +/- 0.00

Eval num_timesteps=3260000, episode_reward=0.18 +/- 0.53

Episode length: 46.10 +/- 11.70

Eval num_timesteps=3270000, episode_reward=0.37 +/- 0.88

Episode length: 42.20 +/- 13.41

# Display

In [ ]:
# Execute Setup
import nbformat
from IPython import get_ipython
with open("02_continuous_world.ipynb", "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)
for cell in notebook.cells:
    if "tags" in cell.metadata and "setup" in cell.metadata.tags:
        exec(cell.source)

env = make_env(render_mode='human')
obs, info = env.reset()

#model = agent.load(best_model_save_path + "best_model.zip")
model = agent.load(best_model_save_path + "/checkpoint_160000_steps.zip")

terminated = False
truncated = False
while not terminated and not truncated:
    
    action, _ = model.predict(obs)
    action = int(action)
    obs, rew, terminated, truncated, info = env.step(action)
    print(f"Action: {action}, Obs: {obs}, Rew: {rew}")
    
env.close()